In [2]:
import pandas as pd
import os
import csv
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
data=[]
with open('image_id.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(['image_id'])
    for filename in os.listdir("gen_samples"):
        data.append(filename)
        writer.writerow(data)
        data=[]
writeFile.close()

df = pd.read_csv('image_id.csv')


class LeafData(Dataset):

    def __init__(self,
                 data,
                 directory,
                 transform=None):
        self.data = data
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # import
        path = os.path.join(self.directory, self.data.iloc[idx]['image_id'])
        image = cv2.imread(path, cv2.IMREAD_COLOR)

        # augmentations
        if self.transform is not None:
            image = self.transform(image=image)['image']

        return image

# dataset
image_dataset = LeafData(data      = df,
                         directory = 'gen_samples',
                         transform = None)

# data loader
image_loader = DataLoader(image_dataset,
                          batch_size  = 1,
                          shuffle     = False,
                          num_workers = 0)

In [4]:
import pathlib
path1 = pathlib.Path('./Image-coco/cover/')
files1 = list(path1.glob('*'))

train_image_list = []

for i in range(len(files1)):

    # read training image
    train_image = cv2.imread(str(files1[i]), cv2.IMREAD_COLOR)
    train_image = torch.from_numpy(train_image).to(torch.long)
    
    # append to lists
    train_image_list.append(train_image)
    

# placeholders
psum    = torch.zeros_like(train_image)
psum_sq = torch.zeros_like(train_image)

i = 0
psum_list = []
psum_sq_list = []

# loop through 1000 sampled images
for inputs in tqdm(image_loader):
    inputs = inputs.to(torch.long)
    # print(inputs.size())
    psum    += inputs.sum(axis        = [0])
    psum_sq += (inputs ** 2).sum(axis = [0])
    i += 1
    if i % 25==0:
        psum_list.append(psum)
        psum_sq_list.append(psum_sq)
        psum    = torch.zeros_like(train_image)
        psum_sq = torch.zeros_like(train_image)

print(len(psum_list))
print(len(psum_sq_list))
print(i)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 469.29it/s]

40
40
1000


In [5]:
# image_size
count = 244 * 164

std_list = []

####### Calculate the Standard Deviation of each set of generated cover images
for i in range(len(psum_list)):
    mean = psum_list[i] / 25
    var = psum_sq_list[i] / 25 - (mean ** 2)
    std = torch.sqrt(var.sum(axis=[0, 1]) / count)
    # print('average pixel-wise std for each channel:  '  + str(std))
    std = torch.mean(std)
    # print('average std:  '  + str(std))
    
    std_list.append(std)
    
print(std_list)

[tensor(27.1471), tensor(21.7544), tensor(14.0521), tensor(17.5488), tensor(22.9270), tensor(29.5459), tensor(18.3266), tensor(14.9391), tensor(28.6792), tensor(18.9467), tensor(16.9008), tensor(28.1648), tensor(30.9998), tensor(15.9014), tensor(16.8295), tensor(26.1731), tensor(14.3156), tensor(16.2218), tensor(21.4377), tensor(11.8613), tensor(29.7708), tensor(16.6423), tensor(36.0550), tensor(18.8303), tensor(16.7913), tensor(22.5471), tensor(17.6061), tensor(16.1859), tensor(14.6325), tensor(22.2143), tensor(15.6117), tensor(16.1569), tensor(15.4818), tensor(13.9603), tensor(14.7105), tensor(28.0563), tensor(8.4409), tensor(13.1589), tensor(18.7084), tensor(22.7228)]


In [6]:
####### Calculate the Standard Deviation of each Traning Image
train_std_list = []

for i in range(len(train_image_list)):
    psum    = torch.tensor([0.0,0.0,0.0])
    psum_sq = torch.tensor([0.0,0.0,0.0])
    
    psum    += train_image_list[i].sum(axis        = [0, 1])
    psum_sq += (train_image_list[i] ** 2).sum(axis = [0, 1])
    
    # mean and std of i-th training image
    train_mean = psum / count
    train_var  = (psum_sq / count) - (train_mean ** 2)
    train_std  = torch.sqrt(train_var)
    # print('pixel-wise average std of training image for each channel:  '  + str(train_std))
    train_std = torch.mean(train_std)
    # print('average std of training image:  '  + str(train_std))

    train_std_list.append(train_std)

print(train_std_list)

[tensor(63.7685), tensor(40.8997), tensor(37.1177), tensor(50.4098), tensor(66.1276), tensor(36.5928), tensor(25.2818), tensor(32.0097), tensor(63.8315), tensor(56.9703), tensor(21.2125), tensor(49.6604), tensor(50.9792), tensor(16.5515), tensor(54.1654), tensor(44.8908), tensor(38.4053), tensor(46.9346), tensor(38.9547), tensor(33.5067), tensor(47.8889), tensor(24.9307), tensor(44.2016), tensor(32.5984), tensor(40.3982), tensor(57.5092), tensor(35.2511), tensor(24.8039), tensor(54.9696), tensor(45.1007), tensor(44.5546), tensor(43.7827), tensor(31.5404), tensor(33.5104), tensor(25.2575), tensor(43.9963), tensor(35.9467), tensor(70.8452), tensor(32.2612), tensor(73.6799)]


In [7]:
####### Normalize to get diversity score
div_score_list = []
for i in range(len(std_list)):
    div_score = std_list[i] / train_std_list[i]
    print('Diversity Score:  '  + str(div_score))
    div_score_list.append(div_score)

print('Average Diversity Score:  '+str(sum(div_score_list)/len(div_score_list)))

Diversity Score:  tensor(0.4257)
Diversity Score:  tensor(0.5319)
Diversity Score:  tensor(0.3786)
Diversity Score:  tensor(0.3481)
Diversity Score:  tensor(0.3467)
Diversity Score:  tensor(0.8074)
Diversity Score:  tensor(0.7249)
Diversity Score:  tensor(0.4667)
Diversity Score:  tensor(0.4493)
Diversity Score:  tensor(0.3326)
Diversity Score:  tensor(0.7967)
Diversity Score:  tensor(0.5671)
Diversity Score:  tensor(0.6081)
Diversity Score:  tensor(0.9607)
Diversity Score:  tensor(0.3107)
Diversity Score:  tensor(0.5830)
Diversity Score:  tensor(0.3727)
Diversity Score:  tensor(0.3456)
Diversity Score:  tensor(0.5503)
Diversity Score:  tensor(0.3540)
Diversity Score:  tensor(0.6217)
Diversity Score:  tensor(0.6675)
Diversity Score:  tensor(0.8157)
Diversity Score:  tensor(0.5776)
Diversity Score:  tensor(0.4156)
Diversity Score:  tensor(0.3921)
Diversity Score:  tensor(0.4994)
Diversity Score:  tensor(0.6526)
Diversity Score:  tensor(0.2662)
Diversity Score:  tensor(0.4925)
Diversity 